In [1]:
import tensorflow
tf = tensorflow.compat.v1
tf.disable_eager_execution()

In [ ]:
import pandas as pd
raw_data = pd.read_csv("./Simulator/data/sim_train.csv", index_col="时间戳")
raw_data

,A磨煤机电流,A磨煤机进口与分离器出口差压,A磨煤机入口一次风风量2,A磨煤机入口热风隔绝门前一次风压力,A磨煤机入口密封风压力,A磨煤机密封风与一次风差压,A磨出口风粉混合物温度,3#A给煤机煤量(瞬时）,A磨煤机入口一次风风温,A磨煤机出口风压,...,过热器右侧一级喷水减温调节电动门开度反馈,过热器左侧一级喷水减温调节电动门开度反馈,过热器左侧二级喷水减温调节电动门开度反馈,过热器右侧二级喷水减温调节电动门开度反馈,引风机动叶开度反馈,一次风机入口开度反馈,送风机动叶开度反馈,一次风机速度反馈,左侧空预器冷端温度调节阀开度反馈,右侧空预器冷端温度调节阀开度反馈
时间戳,,,,,,,,,,,,,,,,,,,,,
1551283560,0.5770,0.5566,0.8461,0.6910,0.6603,0.5998,0.7169,0.7715,0.5454,0.2397,...,0.3886,0.0,0.0018,0.4116,0.2240,0.9972,0.0511,0.9148,0.0043,0.0036
1551283620,0.5625,0.5729,0.8584,0.6967,0.6673,0.5952,0.7157,0.7733,0.5420,0.2424,...,0.3886,0.0,0.0018,0.3981,0.2161,0.9973,0.0510,0.9158,0.0043,0.0036
1551283680,0.5738,0.5643,0.8563,0.6901,0.6631,0.5972,0.7136,0.7733,0.5408,0.2425,...,0.3886,0.0,0.0018,0.3710,0.2243,0.9973,0.0504,0.9130,0.0043,0.0036
1551283740,0.5763,0.5631,0.8490,0.6922,0.6619,0.5951,0.7124,0.7735,0.5408,0.2420,...,0.3886,0.0,0.0018,0.3710,0.2240,0.9973,0.0502,0.9147,0.0043,0.0036
1551283800,0.5718,0.5653,0.8597,0.6872,0.6677,0.6046,0.7121,0.7726,0.5399,0.2442,...,0.3886,0.0,0.0018,0.3710,0.2154,0.9972,0.0503,0.9185,0.0274,0.0302
1551283860,0.5557,0.5681,0.8578,0.6970,0.6629,0.5933,0.7122,0.7735,0.5389,0.2457,...,0.3886,0.0,0.0018,0.3710,0.2070,0.9973,0.0502,0.9201,0.0330,0.0309
1551283980,0.5911,0.5548,0.8589,0.6849,0.6579,0.5909,0.7112,0.7748,0.5399,0.2451,...,0.3989,0.0,0.0018,0.3710,0.2238,0.9972,0.0514,0.9175,0.0330,0.0309
1551284040,0.5612,0.5661,0.8583,0.6956,0.6624,0.5954,0.7128,0.7710,0.5374,0.2462,...,0.3989,0.0,0.0018,0.3777,0.2245,0.9973,0.0514,0.9212,0.0757,0.0145
1551284100,0.5535,0.5648,0.8640,0.7010,0.6609,0.5871,0.7120,0.7733,0.5422,0.2448,...,0.4247,0.0,0.0018,0.4068,0.2239,0.9973,0.0513,0.9199,0.0970,0.0145


In [ ]:
cols = raw_data.columns.tolist()
print("origin len: {0}".format(len(cols)))
cols = (cols[51:52] + cols[53:59] + cols [60:61] + cols[62:63] + cols[150:152]   # external input 
    + cols[0:50] + cols[52:53] + cols[122:139]  # Coal Pulverizing state
    + cols[50:51] + cols[59:60] + cols[61:62] + cols[63:101] + cols[112:114] + cols[118:122] + cols[139:145] + cols[146:149] + cols[152:158]    # Burning state
    + cols[101:112] + cols[114:118] + cols[145:146] + cols[149:150] # Steam Circulation state
    + cols[158:173] + cols[196:202] # Coal Pulverizing action
    + cols[173:192]                 # Burning action
    + cols[192:196])                # Steam Circulation action
print("ordered len: {0}".format(len(cols)))


origin len: 202
ordered len: 202


In [2]:
n_inputs = 3
n_neurons = 5

X0 = tf.placeholder(dtype=tf.float32, shape=[None, n_inputs])
X1 = tf.placeholder(dtype=tf.float32, shape=[None, n_inputs])

Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons], dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons, n_neurons], dtype=tf.float32))
b = tf.Variable(tf.zeros([1, n_neurons], dtype=tf.float32))

Y0 = tf.tanh(tf.matmul(X0, Wx) + b)
Y1 = tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1, Wx) + b)

init = tf.global_variables_initializer()

In [3]:
import numpy as np

# Mini-batch:  4 instances
X0_batch = np.array([
    [0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0, 1]
])
X1_batch = np.array([
    [9, 8, 7], [0, 0, 0], [6, 5, 4], [3, 2, 1]
])

在之前定义tf.variable里的W时只是定义了这个变量值初始化的方式的，但并没有真正对其初始化，直到运行了tf.global_variables_initializer().run()之后才完成了变量的初始化。
sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch}) 的意思是执行[Y0, Y1]函数，执行时将其中的X0和X1替换为X0_batch, X1_batch。
下面的函数只是执行了一步，还并没有开始训练

In [4]:
with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})

In [5]:
print(Y0_val)
print(Y1_val)

[[ 0.99721074  0.9999335   0.48385316 -0.96940815 -0.3043463 ]
 [ 1.          1.          0.9999827  -1.         -0.9858921 ]
 [ 1.          1.          1.         -1.         -0.99981076]
 [ 1.         -1.          1.         -1.         -0.80549014]]
[[ 1.          1.          1.         -1.         -0.99999636]
 [-0.99969256 -0.89230347 -0.7638404  -0.25740027 -0.9329769 ]
 [ 1.          0.99966717  1.         -1.         -0.99997884]
 [ 0.99997514 -0.7768876   0.99455863 -1.         -0.99953794]]


In [6]:
X0 = tf.placeholder(dtype=tf.float32, shape=[None, n_inputs])
X1 = tf.placeholder(dtype=tf.float32, shape=[None, n_inputs])

# basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
basic_cell = tensorflow.keras.layers.SimpleRNNCell(units=n_neurons)
output_seqs, states = tf.nn.static_rnn(cell=basic_cell, inputs=[X0, X1], dtype=tf.float32)

Y0, Y1 = output_seqs

Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API


上面这种方法虽然可行，但显然当时间迭代长的时候是一件不那么容易的事情，所以，我们换一种做法。

tf.transpose(X, perm=[1, 0, 2]) 是一个张量转置的函数，在这里就是将第一维和第二维转置，第三维保持不变。交换维度后，第一个时间迭代将称为第一个维度。unstack函数将抽取第一个维度的张良；stack函数合并所有输出张量为一个张量

In [7]:
n_steps = 2
n_inputs = 3
n_neurons = 5

In [8]:
tf.reset_default_graph()

X = tf.placeholder(dtype=tf.float32, shape=[None, n_steps, n_inputs])
X_seqs = tf.unstack(tf.transpose(X, perm=[1, 0, 2]))
basic_cell = tensorflow.keras.layers.SimpleRNNCell(units=n_neurons)
# basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.nn.static_rnn(cell=basic_cell, inputs=X_seqs, dtype=tf.float32)

outputs = tf.transpose(tf.stack(output_seqs), perm=[1, 0, 2])   # shape [None, n_steps, n_neurons]

In [9]:
X_batch = np.array([
        # t = 0      t = 1 
        [[0, 1, 2], [9, 8, 7]], # instance 1
        [[3, 4, 5], [0, 0, 0]], # instance 2
        [[6, 7, 8], [6, 5, 4]], # instance 3
        [[9, 0, 1], [3, 2, 1]], # instance 4
    ])

In [10]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    outputs_val = outputs.eval(feed_dict={X: X_batch})

In [11]:
print(outputs_val)

[[[-0.9437216  -0.9688752  -0.9254175  -0.72729063  0.6664808 ]
  [-1.         -0.9916848  -0.99999934 -0.9999994   1.        ]]

 [[-1.         -0.99915594 -0.999774   -0.997885    0.9998903 ]
  [ 0.28398967  0.7335063  -0.45975932 -0.81245774 -0.9186842 ]]

 [[-1.         -0.9999774  -0.99999934 -0.99998575  1.        ]
  [-1.         -0.5456308  -0.9998378  -0.9999411   0.99998224]]

 [[-0.9999995   0.9999763  -0.8636595  -0.9996743   0.9999993 ]
  [-0.9999479   0.97578293 -0.93021363 -0.95961726  0.99363333]]]


静态展开很有可能会造成内存溢出，更好的做法应该是使用通过实践的动态展开，即dynamic_run

In [15]:
tf.reset_default_graph()

X = tf.placeholder(dtype=tf.float32, shape=[None, n_steps, n_inputs])

basic_cell = tensorflow.keras.layers.SimpleRNNCell(units=n_neurons)
output_seqs, states = tf.nn.dynamic_rnn(cell=basic_cell, inputs=X, dtype=tf.float32)
# output_seqs, states = tensorflow.keras.layers.RNN(cell=basic_cell, inputs=X, dtype=tf.float32)

outputs = tf.transpose(tf.stack(output_seqs), perm=[1, 0, 2])   # shape [None, n_steps, n_neurons]

In [16]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    outputs_val = outputs.eval(feed_dict={X: X_batch})
print(outputs_val)

[[[-0.04931172  0.79321074  0.8753438   0.91418236 -0.4044742 ]
  [-0.41395688  0.8886208   0.9982269   0.99997056  0.24415475]
  [-0.6812283   0.94144726  0.99997646  1.          0.72936165]
  [-0.96296126 -0.3227796   0.9312403   0.9999824   0.9999989 ]]

 [[-0.9685134   0.11808459  0.9999349   1.          0.96396995]
  [-0.62497747 -0.3147351   0.29549137  0.8355327  -0.72050816]
  [-0.8401544  -0.7986587   0.9979395   0.9999997   0.5194646 ]
  [-0.6585177  -0.98426414  0.8387527   0.99621475  0.5303947 ]]]
